In [1]:
from sentence_transformers import SentenceTransformer
from docuverse.utils import open_stream, read_config_file
import pickle
from tqdm.autonotebook import tqdm, trange
from docuverse.utils.text_tiler import TextTiler
from docuverse.engines.search_data import SearchData
from docuverse.engines.data_template import DataTemplate
import os

/home/raduf/miniforge3/envs/docuverse/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
# model_name = "thenlper/gte-small"
model_name = "/home/raduf/mnlp-models-rag/06302024/slate.30m.english.rtrvr"
model = SentenceTransformer(model_name)
max_size = 512
stride = 100
tiler = TextTiler(max_doc_size=max_size, stride=stride, tokenizer=model.tokenizer, aligned_on_sentences=False)

In [3]:

# datadir="data/clapnq_small"
datadir="data/long_example"

In [4]:
file = "Long programmatic PDF.md"
if file == "passages.tsv":
     import pandas as pd
     data = pd.read_csv(os.path.join(datadir, "passages.tsv"), sep="\t")
     sents = data['text']
elif file == "Long programmatic PDF.md":
    sents = ["".join(open_stream(os.path.join(datadir, "Long programmatic PDF.md")).readlines())]

In [5]:
sents[0][:1000]

"The Project Gutenberg EBook of The Brothers Karamazov by  Fyodor Dostoyevsky\n\nThis ebook is for the use of anyone anywhere in the United States  and most other parts of the world at no cost and with almost no  restrictions whatsoever. You may copy it, give it away or re-use  it under the terms of the Project Gutenberg License included with  this eBook or online at http://www.gutenberg.org/license. If you  are not located in the United States, you'll have to check the laws  of the country where you are located before using this ebook.\n\nTitle: The Brothers Karamazov\n\nAuthor: Fyodor Dostoyevsky\n\nRelease Date: February 12, 2009 [Ebook 28054]\n\nRelease Date: February 12, 2009 [Ebook 28054]\n\nLanguage: English\n\n***START OF THE PROJECT GUTENBERG EBOOK  THE BROTHERS KARAMAZOV***\n\nThe Brothers Karamazov  Translated from the Russian of  Fyodor Dostoyevsky  by Constance Garnett  The Lowell Press  New York\n\n\n## Part I\n\n## Book I. The History Of A Family\n\n## Chapter I. Fyodor 

In [6]:
tiler.tokenizer.cls_token

'<s>'

In [7]:
tiled_data = [tiler.create_tiles(i, txt, '', max_doc_size=512, stride=100, title_handling="none") for i, txt in enumerate(sents)]

In [8]:
tiled_data[0][:10]

[{'id': '0-0-2035',
  'text': "The Project Gutenberg EBook of The Brothers Karamazov by Fyodor Dostoyevsky\n\nThis ebook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at URL. If you are not located in the United States, you'll have to check the laws of the country where you are located before using this ebook.\n\nTitle: The Brothers Karamazov\n\nAuthor: Fyodor Dostoyevsky\n\nRelease Date: February 12, 2009 [Ebook 28054]\n\nRelease Date: February 12, 2009 [Ebook 28054]\n\nLanguage: English\n\n***START OF THE PROJECT GUTENBERG EBOOK THE BROTHERS KARAMAZOV***\n\nThe Brothers Karamazov Translated from the Russian of Fyodor Dostoyevsky by Constance Garnett The Lowell Press New York\n\n\n## Part I\n\n## Book I. The History Of A Family\n\n## Chapter I. Fyodor Pavlovitch K

# Run the tiler in parallel

In [51]:
from docuverse.utils import parallel_process

def proc_text(item):
     res = tiler.create_tiles(item['id'], item['text'], item['title'])
     item['result'] = res
     return item

res = parallel_process(proc_text, [{'id': i, 'text': txt, 'title': f"Title for item {i}"} for i, txt in enumerate(sents[:100])], num_threads=10)

Processing result:::   0%|          | 0/100 [00:00<?, ?it/s]

In [52]:
res

[{'id': 0,
  'text': 'Egocentrism is the inability to differentiate between self and other . More specifically , it is the inability to untangle subjective schemas from objective reality ; an inability to understand or assume any perspective other than their own .',
  'title': 'Title for item 0',
  'result': [{'id': '0-0-242',
    'text': 'Title for item 0\nEgocentrism is the inability to differentiate between self and other . More specifically , it is the inability to untangle subjective schemas from objective reality ; an inability to understand or assume any perspective other than their own .'}]},
 {'id': 1,
  'text': "Although egocentrism and narcissism appear similar , they are not the same . A person who is egocentric believes they are the center of attention , like a narcissist , but does not receive gratification by one 's own admiration . Both egotists and narcissists are people whose egos are greatly influenced by the approval of others , while for egocentrists this may or ma